<!--NAVIGATION-->


<a href="https://colab.research.google.com/github/UAH-EPS-GII-GINFOADE-SSOO/LabSSOO/blob/main/p1/notebooks/notebookSeñales.ipynb" target="_blank" rel="noopener noreferrer"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

<h1>Comunicación entre procesos</h1>

<div style="font-size: 17px">

La **comunicación entre procesos** (*IPC* por sus siglas en inglés ***Inter-Process Communication***) en *Linux* es un conjunto de mecanismos que permiten que dos o más procesos se intercambien datos, se coordinen o se sincronicen mientras se ejecutan. Dado que cada proceso en *Linux* tiene su propio espacio de memoria, no pueden compartir información directamente, por lo que necesitan mecanismos específicos para comunicarse.

Existen diferentes métodos de comunicación entre procesos, cada uno adecuado para distintos tipos de interacción. Entre los mecanismos más comunes se encuentran las **tuberías**, utilizadas para enviar información entre procesos (se explicarán más adelante en otra sección), y las **señales**, empleadas para notificar eventos o cambios de estado.

</div>

<h2>Señales</h2>

<div style="font-size: 17px">

Las **señales** son una forma de comunicación entre procesos que permite enviar órdenes simples, como finalizar, pausar o reanudar un proceso. Cada señal está identificada por un nombre simbólico (como <code>SIGKILL</code>) y un número asociado. Cuando un proceso recibe una señal, puede reaccionar según su programación, o en algunos casos, no puede evitar la acción.  

Se puede pensar en una **señal** en *Linux* como una forma de "interrupción de *software*". Tiene en común con las interrupciones hardware las siguientes características:  

- Son **asíncronos**: pueden ocurrir en cualquier momento.
- Interrumpen el flujo normal de ejecución para que se realice una acción inmediata.
- Requieren que el sistema (o el proceso) **reaccione** de alguna forma.

<h3>Comparación: Interrupción de hardware VS Señal</h3>

| Característica           | Interrupción de hardware         | Señal (software)                      |
|--------------------------|----------------------------------|--------------------------------------|
| **Origen**               | Dispositivo físico (teclado, red, reloj, etc.) | Proceso, usuario o *kernel*             |
| **Destino**              | *CPU* o sistema operativo          | Un proceso específico                |
| **Ejemplo**              | Pulsar una tecla                | <code>SIGINT</code> (cuando presionas <code>Ctrl + C</code>) |
| **Manejador (handler)**  | Rutina de atención a interrupciones (*ISR*) | Rutina de manejo de señales (<code>trap</code>, en *Bash*) |
| **Puede ser ignorada**   | No                               | Algunas sí (<code>SIGTERM</code>), otras no (<code>SIGKILL</code>) |

Las señales más comunes son:

| Número | Señal    | Descripción                                                                         |
|--------|----------|-------------------------------------------------------------------------------------|
| 0      | `EXIT`   | Señal que se envía cuando un *script* finaliza. |
| 1      | `SIGHUP` | Señal de "cuelgue" o desconexión. Reinicia el proceso o recarga su configuración.    |
| 2      | `SIGINT` | Señal de interrupción. Generalmente activada al presionar `Ctrl + C`.                |
| 9      | `SIGKILL`| Termina el proceso de inmediato. No puede ser ignorada ni manejada.   |
| 15     | `SIGTERM`| Señal de terminación estándar, permite al proceso finalizar de forma ordenada.       |
| 17     | `SIGSTOP`| Pausa la ejecución del proceso. No puede ser ignorada por el proceso.                |
| 18     | `SIGCONT`| Reanuda la ejecución de un proceso que fue pausado con `SIGSTOP`.                    |

Para mandar estas señales, existen los siguientes comandos:

  - **<code>kill PID</code>**: Envía una señal al proceso identificado con el *PID* correspondiente. Aunque su nombre sugiere que solo se usa para terminar procesos, en realidad puede enviar distintas señales.
    - <code>-NUM</code> o <code>-NOMBRE</code>: Manda una señal concreta (por ejemplo, <code>-9</code> o <code>-SIGKILL</code> para mandar <code>SIGKILL</code>). Si no se indica, se envía por defecto <code>SIGTERM</code>.
    - <code>-l</code>: Esta opción lista todas las señales disponibles, mostrando tanto su número como su nombre. En caso de pasar como argumento un número, mostrará el nombre correspondiente a este número y viceversa

**Ejemplo**: Primero se muestra el nombre de la señal asociada al número 18, y luego se listan todas las señales:
</div>


In [2]:
%%shell
echo "El nombre de la señal con el número 18 es:"
kill -l 18
echo "Estas son todas las señales:"
kill -l

El nombre de la señal con el número 18 es:
CONT
Estas son todas las señales:
 1) SIGHUP	 2) SIGINT	 3) SIGQUIT	 4) SIGILL	 5) SIGTRAP
 6) SIGABRT	 7) SIGBUS	 8) SIGFPE	 9) SIGKILL	10) SIGUSR1
11) SIGSEGV	12) SIGUSR2	13) SIGPIPE	14) SIGALRM	15) SIGTERM
16) SIGSTKFLT	17) SIGCHLD	18) SIGCONT	19) SIGSTOP	20) SIGTSTP
21) SIGTTIN	22) SIGTTOU	23) SIGURG	24) SIGXCPU	25) SIGXFSZ
26) SIGVTALRM	27) SIGPROF	28) SIGWINCH	29) SIGIO	30) SIGPWR
31) SIGSYS	34) SIGRTMIN	35) SIGRTMIN+1	36) SIGRTMIN+2	37) SIGRTMIN+3
38) SIGRTMIN+4	39) SIGRTMIN+5	40) SIGRTMIN+6	41) SIGRTMIN+7	42) SIGRTMIN+8
43) SIGRTMIN+9	44) SIGRTMIN+10	45) SIGRTMIN+11	46) SIGRTMIN+12	47) SIGRTMIN+13
48) SIGRTMIN+14	49) SIGRTMIN+15	50) SIGRTMAX-14	51) SIGRTMAX-13	52) SIGRTMAX-12
53) SIGRTMAX-11	54) SIGRTMAX-10	55) SIGRTMAX-9	56) SIGRTMAX-8	57) SIGRTMAX-7
58) SIGRTMAX-6	59) SIGRTMAX-5	60) SIGRTMAX-4	61) SIGRTMAX-3	62) SIGRTMAX-2
63) SIGRTMAX-1	64) SIGRTMAX	


<div style="font-size: 17px">

  - **<code>killall nombre</code>**: Envía señales a todos los procesos que coincidan con un nombre específico, sin necesidad de conocer su *PID*.

**Ejemplo**: Se crea un proceso <code>sleep</code>, y se envía una señal a todos los procesos <code>sleep</code> para que acaben:

In [3]:
%%shell
sleep 5 &
killall sleep

<div style="font-size: 17px">

Para capturar señales:

- **<code>trap</code>**: Permite ejecutar un comando específico cuando llega una señal determinada.

**Ejemplo**: Se establece una acción con <code>trap</code> (<code>echo</code> muestra un mensaje) que solo se ejecuta al recibir la señal <code>EXIT</code>:
</div>

In [ ]:
%%shell
trap 'echo "Script is exiting!"' EXIT
sleep 2
echo "Done sleeping"
exit

Done sleeping
Script is exiting!


<div style="font-size: 17px">

Pese a que la orden <code>trap</code> esté la primera, el comando <code>echo</code> no se ejecuta hasta que detecte la señal de <code>EXIT</code>, es decir, acabe el programa.
</div>

<h3>Ejercicio</h3>

<div style="font-size: 17px">

El siguiente *shell script*, denominado <code>scriptprueba.sh</code>, que muestra por pantalla la frase "Soy solo un bucle" cada segundo 10 veces. Usando señales, añade en la línea 2 del *script* una orden para que cuando se mate el proceso muestre un mensaje:
```
1. #!/bin/bash
2.
3. for i in {1..10}
4. do
5.     echo "Soy solo un bucle"
6.     sleep 1
7. done
```

</div>

In [ ]:
%%shell
echo '#!/bin/bash' >> scriptprueba.sh
echo '' >> scriptprueba.sh #Introduce aquí la orden necesaria
echo 'for i in {1..10}' >> scriptprueba.sh
echo 'do' >> scriptprueba.sh
echo '    echo "Soy solo un bucle"' >> scriptprueba.sh
echo '    sleep 1' >> scriptprueba.sh
echo 'done' >> scriptprueba.sh

bash scriptprueba.sh & #Ejecuta el script en segundo plano
PID=$!
sleep 4
kill -SIGTERM $PID

rm scriptprueba.sh

<div style="font-size: 17px">

Teniendo el *script* anterior, manda la señal necesaria para pararlo a mitad de su ejecución y continuarlo pasados 3 segundos:

</div>

In [ ]:
%%shell
echo '#!/bin/bash' >> scriptprueba.sh
echo 'for i in {1..10}' >> scriptprueba.sh
echo 'do' >> scriptprueba.sh
echo '    echo "Soy solo un bucle"' >> scriptprueba.sh
echo '    sleep 1' >> scriptprueba.sh
echo 'done' >> scriptprueba.sh

bash scriptprueba.sh & #Ejecuta el script en segundo plano
PID=$!
sleep 5
#Borra este comentario e introduce aquí el/los comando/s necesario/s

rm scriptprueba.sh